In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-06-05 14:40:53--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5488 (5.4K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.36K  --.-KB/s    in 0s      

2025-06-05 14:40:53 (33.1 MB/s) - ‘minsearch.py.1’ saved [5488/5488]



In [3]:
import minsearch

In [4]:
import json

In [5]:
with open ('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
q = 'the course has already started, can I still enroll?'

In [11]:
index.fit(documents)

In [12]:
boost = {'question' : 3.0, 'section' : 0.5}
results = index.search(
    query = q,
    filter_dict = {'course' : 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results  = 5
)



In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [47]:
import os
from openai import OpenAI

api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(
    api_key=api_key,
    base_url="https://openrouter.ai/api/v1"
)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[{"role": "user", "content": q}]
)

print(response.choices[0].message.content)


Not necessarily, but in many cases, you can still enroll even if a course has already started — it just depends on the policies of the institution or platform offering the course. Here’s what you should check:

1. **Visit the Course Enrollment Link/Platform**: If the course is available online via a platform like Google Classroom, Canvas, Blackboard, Fleep Q/Course Hero, or another learning system, look to see if enrollment is still open or if there's a late enrollment option.

2. **Check the Course Curriculum**: Often, even if late enrollment isn't directly offered, you can still access the course materials and content by clicking through to the course folder.

3. **Ask Your Teacher/Course Instructor**: Email your instructor or contact the school/program coordinator for guidance. They can tell you whether joining late is possible and if there are any catch-up requirements.

4. **Check for a Course Code**: Sometimes courses require a code to enroll. This might have been shared by the t

In [41]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer ,output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [42]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [43]:
prompt = prompt_template.format(question = q, context = context).strip()

In [44]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer ,output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


In [46]:
response = client.chat.completions.create(
    model="deepseek/deepseek-r1-0528-qwen3-8b:free",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

Yes, even after the course has started, you are still eligible to enroll and submit homeworks. However, be mindful of the deadlines for turning in your final projects.
